### Data preparation and Exploration

In [22]:
import pandas as pd 
import numpy as np
import json
import re
from tqdm import tqdm,tqdm_gui
from tqdm._tqdm_notebook import tqdm_notebook

In [23]:
PATH = './data/'

In [24]:
with open(PATH+'review.json') as f:
    review_data = pd.DataFrame(json.loads(line) for line in f)

In [25]:
with open(PATH+'business.json') as f:
    business_data = pd.DataFrame(json.loads(line) for line in f)

In [26]:
rest_biz = business_data[business_data['categories'].apply(str).str.contains('Restaurants')]

In [27]:
rest_biz = rest_biz.drop(columns=['stars'])

In [28]:
rest_biz.sort_values(by='review_count',ascending=False,inplace=True)

In [29]:
rest_biz.groupby(by=['state'])['review_count'].sum().sort_values(ascending=False)

state
NV     949953
AZ     837216
ON     414411
NC     180487
OH     154726
PA     143283
QC      98978
WI      69050
BW      24934
EDH     23747
IL      22186
SC       5981
MLN      1101
HLD       589
C         168
ELN       117
FIF       110
NYK       101
WLN        87
NY         73
NI         58
WA         40
01         24
PKN        24
ST         24
ESX        11
BY         10
KHL         7
RCC         7
XGL         6
3           5
HH          4
CA          4
WHT         4
ABE         3
ZET         3
Name: review_count, dtype: int64

In [30]:
preprocessed_data = rest_biz.merge(review_data,how='inner')
preprocessed_data = preprocessed_data[preprocessed_data['state'].isin(['NV'])]

In [31]:
preprocessed_data.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,review_count,state,cool,date,funny,review_id,stars,text,useful,user_id
0,3655 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'HasTV': False, 'Noise...",4JNXUYY8wbaaDmk3BPzlWw,"[French, Steakhouses, Breakfast & Brunch, Rest...",Las Vegas,"{'Monday': '7:00-23:00', 'Tuesday': '7:00-23:0...",1,36.112827,-115.172581,Mon Ami Gabi,...,6979,NV,0,2011-02-22,0,WE9eUYf5EV8AxJjl8QZRtA,5,"Very chic. Although, the menu items doesnt SCR...",0,K6FpHYwcJYznoXXu8ySZHw
1,3655 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'HasTV': False, 'Noise...",4JNXUYY8wbaaDmk3BPzlWw,"[French, Steakhouses, Breakfast & Brunch, Rest...",Las Vegas,"{'Monday': '7:00-23:00', 'Tuesday': '7:00-23:0...",1,36.112827,-115.172581,Mon Ami Gabi,...,6979,NV,0,2015-04-15,0,7Fl41hKa0wjO3TlvHKD3lw,4,Cute french bistro with great service. I came ...,1,3SGQKsO1J-jcRIp3WNxCeA
2,3655 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'HasTV': False, 'Noise...",4JNXUYY8wbaaDmk3BPzlWw,"[French, Steakhouses, Breakfast & Brunch, Rest...",Las Vegas,"{'Monday': '7:00-23:00', 'Tuesday': '7:00-23:0...",1,36.112827,-115.172581,Mon Ami Gabi,...,6979,NV,0,2013-11-11,0,Jm4iOfsltS3T59puoV6r8Q,5,"The food is very good, the prices fair and the...",0,LPT8XlpXlHGAp0Ri4Hu4Rw
3,3655 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'HasTV': False, 'Noise...",4JNXUYY8wbaaDmk3BPzlWw,"[French, Steakhouses, Breakfast & Brunch, Rest...",Las Vegas,"{'Monday': '7:00-23:00', 'Tuesday': '7:00-23:0...",1,36.112827,-115.172581,Mon Ami Gabi,...,6979,NV,3,2015-12-28,2,m6lwvXPCdpKHVp05Bjkhlw,5,One of my favorite go-to brunch spots on the L...,8,3NnPbhmv_vEfPTBp2pnn9Q
4,3655 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'HasTV': False, 'Noise...",4JNXUYY8wbaaDmk3BPzlWw,"[French, Steakhouses, Breakfast & Brunch, Rest...",Las Vegas,"{'Monday': '7:00-23:00', 'Tuesday': '7:00-23:0...",1,36.112827,-115.172581,Mon Ami Gabi,...,6979,NV,0,2015-09-16,0,nkb40j7kBe2qAci1xpyd3g,5,Simply amazing steak and frites. Got the blue ...,0,xl4rsQqpibUNhR8Jqxp4OQ


In [32]:
date_sorted = preprocessed_data.sort_values(by='date',ascending=False)

In [33]:
data_20150101 = date_sorted[date_sorted['date']>'2017-01-01']

In [34]:
data_20150101 = data_20150101.reset_index().drop('index',axis=1)

In [35]:
data_20150101.groupby(by=['state'])['state'].count()

state
NV    126225
Name: state, dtype: int64

In [36]:
data_20150101.shape

(126225, 22)

In [39]:
preprocessed_dataviz = data_20150101.loc[:,['business_id','user_id','text']]

In [40]:
# The data for this analysis include restaurant reviews across two states Nevada and Arizona for the past 2.5 years

In [41]:
preprocessed_dataviz.to_feather('./data/preprocessed_data')

### Tokenization 

In [7]:
import pandas as pd
import spacy
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim import models,corpora
from tqdm import tqdm,tqdm_gui
from tqdm._tqdm_notebook import tqdm_notebook

In [3]:
preprocessed_data = pd.read_feather('preprocessed_data')
preprocessed_data = preprocessed_data.reset_index().drop('index',axis=1)

In [5]:
nlp = spacy.load('en')

1. Convert the reviews into sentence streams
2. Construct a bigram model
3. Using a bigram model, construct 

Here, we want to create a stream of sentences. And use that to create our bigram model.

In [8]:
doc = nlp.pipe(preprocessed_data['text'],n_threads=-1)
text_out = []
removal = []
for review in tqdm(doc):
    review_sent = []
    for sent in review.sents:
        sentence = []
        for token in sent:        
            if token.is_alpha and token.is_stop == False and token.pos_ not in removal:
                lemma = token.lemma_
                sentence.append(lemma)
        review_sent.append(sentence)
    text_out.append(review_sent)

126225it [24:55, 84.42it/s]


In [38]:
total_text = []
for text in text_out:
    total_text +=text
common_terms = ['by' ,'in', 'of' ,'on' ,'or', 'to','the']
bigram_model = Phrases(total_text,scoring='npmi',common_terms=common_terms,threshold=0.5,min_count=100)

INFO : collecting all words and their counts
INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO : PROGRESS: at sentence #10000, processed 57199 words and 38175 word types
INFO : PROGRESS: at sentence #20000, processed 115696 words and 68327 word types
INFO : PROGRESS: at sentence #30000, processed 173238 words and 94673 word types
INFO : PROGRESS: at sentence #40000, processed 231888 words and 119857 word types
INFO : PROGRESS: at sentence #50000, processed 291563 words and 143556 word types
INFO : PROGRESS: at sentence #60000, processed 350225 words and 165111 word types
INFO : PROGRESS: at sentence #70000, processed 409282 words and 186121 word types
INFO : PROGRESS: at sentence #80000, processed 467365 words and 205914 word types
INFO : PROGRESS: at sentence #90000, processed 526147 words and 225305 word types
INFO : PROGRESS: at sentence #100000, processed 584434 words and 244128 word types
INFO : PROGRESS: at sentence #110000, processed 642385 words and 26214

INFO : PROGRESS: at sentence #980000, processed 5746383 words and 1298087 word types
INFO : PROGRESS: at sentence #990000, processed 5804679 words and 1307425 word types
INFO : PROGRESS: at sentence #1000000, processed 5865678 words and 1317433 word types
INFO : PROGRESS: at sentence #1010000, processed 5926033 words and 1327325 word types
INFO : PROGRESS: at sentence #1020000, processed 5984415 words and 1336298 word types
INFO : PROGRESS: at sentence #1030000, processed 6043695 words and 1345358 word types
INFO : PROGRESS: at sentence #1040000, processed 6102974 words and 1354640 word types
INFO : collected 1361489 word types from a corpus of 6146670 words (unigram + bigrams) and 1047276 sentences
INFO : using 1361489 counts as vocab in Phrases<0 vocab, min_count=100, threshold=0.5, max_vocab_size=40000000>


1. Words at review level

In [39]:
processed_review =[]
for text in text_out:
    review =[]
    for t in text:
        review +=t
    processed_review.append(review)
processed_bigram_reviews = [ bigram_model[review] for review in tqdm(processed_review)]


  0%|          | 0/126225 [00:00<?, ?it/s]/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")

 54%|█████▎    | 67571/126225 [00:13<00:11, 5037.45it/s]


100%|██████████| 126225/126225 [00:25<00:00, 4973.78it/s]

In [40]:
processed_bigram_reviews[100]

['ichiza',
 'good',
 'the',
 'thing',
 '-PRON-',
 'like',
 'wagyu',
 'yakitori',
 '-PRON-',
 'order',
 'yakitori',
 'chicken',
 'thigh',
 'wagyu',
 'tongue',
 'ok',
 'wagyu_beef',
 'good',
 'pork_belly',
 'meh',
 'duck',
 'meh',
 'grill',
 'octopus',
 'ok',
 '-PRON-',
 'will',
 'come']

In [41]:
# filtering for stop words
from spacy.lang.en.stop_words import STOP_WORDS 
STOP_WORDS

{'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'front',
 'full',
 'further',
 'get',
 'give',
 'g

In [51]:
final_bigrams = []
stopwordlist = list(STOP_WORDS)
for r in tqdm(processed_bigram_reviews):
    review = []
    for word in r:
        if word not in stopwordlist and word != '-PRON-':
            review.append(word)
    final_bigrams.append(review)
            
        


 60%|██████    | 75897/126225 [00:13<00:09, 5423.59it/s]


100%|██████████| 126225/126225 [00:23<00:00, 5400.51it/s]

In [53]:
final_bigrams[100]

['ichiza',
 'good',
 'thing',
 'like',
 'wagyu',
 'yakitori',
 'order',
 'yakitori',
 'chicken',
 'thigh',
 'wagyu',
 'tongue',
 'ok',
 'wagyu_beef',
 'good',
 'pork_belly',
 'meh',
 'duck',
 'meh',
 'grill',
 'octopus',
 'ok',
 'come']

In [54]:
dictionary = corpora.Dictionary(final_bigrams)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tqdm(final_bigrams)]

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : adding document #10000 to Dictionary(18314 unique tokens: ['amazing', 'bartender', 'busy', 'butt', 'customer']...)
INFO : adding document #20000 to Dictionary(26863 unique tokens: ['amazing', 'bartender', 'busy', 'butt', 'customer']...)
INFO : adding document #30000 to Dictionary(34093 unique tokens: ['amazing', 'bartender', 'busy', 'butt', 'customer']...)
INFO : adding document #40000 to Dictionary(40726 unique tokens: ['amazing', 'bartender', 'busy', 'butt', 'customer']...)
INFO : adding document #50000 to Dictionary(47229 unique tokens: ['amazing', 'bartender', 'busy', 'butt', 'customer']...)
INFO : adding document #60000 to Dictionary(52963 unique tokens: ['amazing', 'bartender', 'busy', 'butt', 'customer']...)
INFO : adding document #70000 to Dictionary(58508 unique tokens: ['amazing', 'bartender', 'busy', 'butt', 'customer']...)
INFO : adding document #80000 to Dictionary(63699 unique tokens: ['amazing', 'bartend

In [55]:
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.DEBUG)

In [56]:
##### Step-3 : Define multicore lda model and enjoy!!!
num_topics =50
Lda = models.LdaMulticore
lda= Lda(doc_term_matrix, num_topics=num_topics,id2word = dictionary, 
         passes=20,chunksize=4000,random_state=42)

INFO : using symmetric alpha at 0.02
INFO : using symmetric eta at 0.02
INFO : using serial LDA version on this node
INFO : running online LDA training, 50 topics, 20 passes over the supplied corpus of 126225 documents, updating every 140000 documents, evaluating every ~126225 documents, iterating 50x with a convergence threshold of 0.001000
INFO : training LDA model using 35 processes
DEBUG : worker process entering E-step loop
DEBUG : getting a new job
DEBUG : getting a new job
DEBUG : worker process entering E-step loop
DEBUG : worker process entering E-step loop
DEBUG : getting a new job
DEBUG : getting a new job
DEBUG : worker process entering E-step loop
DEBUG : worker process entering E-step loop
DEBUG : getting a new job
DEBUG : worker process entering E-step loop
DEBUG : getting a new job
DEBUG : worker process entering E-step loop
DEBUG : worker process entering E-step loop
DEBUG : getting a new job
DEBUG : getting a new job
DEBUG : worker process entering E-step loop
DEBUG :

DEBUG : getting a new job
DEBUG : result put
DEBUG : processing chunk #17 of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : processing chunk #18 of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : 2146/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : processing chunk #19 of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : result put
DEBUG : getting a new job
DEBUG : processing chunk #20 of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : 2133/4000 documents converged within 50 iterations
DEBUG : getting a new job
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : 2087/4000 documents converged within 50 iterations
DEBUG : processing chunk #21 of 4000 documents
DEBUG : processed chunk, queuing the result
DEBUG : performing inference on a chunk of 4000 documents
DEBUG

INFO : PROGRESS: pass 1, dispatched chunk #10 = documents up to #44000/126225, outstanding queue size 11
INFO : PROGRESS: pass 1, dispatched chunk #11 = documents up to #48000/126225, outstanding queue size 12
INFO : PROGRESS: pass 1, dispatched chunk #12 = documents up to #52000/126225, outstanding queue size 13
INFO : PROGRESS: pass 1, dispatched chunk #13 = documents up to #56000/126225, outstanding queue size 14
DEBUG : processing chunk #0 of 4000 documents
INFO : PROGRESS: pass 1, dispatched chunk #14 = documents up to #60000/126225, outstanding queue size 15
DEBUG : performing inference on a chunk of 4000 documents
INFO : PROGRESS: pass 1, dispatched chunk #15 = documents up to #64000/126225, outstanding queue size 16
INFO : PROGRESS: pass 1, dispatched chunk #16 = documents up to #68000/126225, outstanding queue size 17
INFO : PROGRESS: pass 1, dispatched chunk #17 = documents up to #72000/126225, outstanding queue size 18
INFO : PROGRESS: pass 1, dispatched chunk #18 = document

DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : 2593/4000 documents converged within 50 iterations
DEBUG : result put
DEBUG : processed chunk, queuing the result
DEBUG : getting a new job
DEBUG : 2590/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : 2558/4000 documents converged within 50 iterations
DEBUG : result put
DEBUG : processed chunk, queuing the result
DEBUG : getting a new job
DEBUG : 2612/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : 2567/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : 2617/4000 documents converged within 50 iterations
DEBUG : 2584/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : r

DEBUG : getting a new job
DEBUG : processed chunk, queuing the result
DEBUG : processing chunk #12 of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : result put
DEBUG : 2673/4000 documents converged within 50 iterations
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : processing chunk #13 of 4000 documents
DEBUG : processing chunk #14 of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : getting a new job
DEBUG : processing chunk #15 of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : 2667/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : processing chunk #16 of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : getting a new job
DEBUG : 2615/4000 documents converged within 50 iterations
DEBUG

...

DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : 3486/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : 3541/4000 documents converged within 50 iterations
DEBUG : result put
DEBUG : processed chunk, queuing the result
DEBUG : getting a new job
DEBUG : 1943/2225 documents converged within 50 iterations
DEBUG : result put
DEBUG : processed chunk, queuing the result
DEBUG : getting a new job
DEBUG : 3519/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : 3523/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : 3487/4000 documents converged within 50 iterations
DEBUG : result put
DEBUG : processed chunk, queuing the result
DEBUG : getting a new job
DEBUG : 3475/4000 docum

DEBUG : 3554/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : processing chunk #15 of 4000 documents
DEBUG : getting a new job
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : 3563/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : processing chunk #16 of 4000 documents
DEBUG : getting a new job
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : 3564/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : processing chunk #17 of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : 3525/4000 documents converged within 50 iterations
DEBUG : processing chunk #18 of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : gettin

INFO : PROGRESS: pass 18, dispatched chunk #13 = documents up to #56000/126225, outstanding queue size 14
DEBUG : processing chunk #1 of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
INFO : PROGRESS: pass 18, dispatched chunk #14 = documents up to #60000/126225, outstanding queue size 15
INFO : PROGRESS: pass 18, dispatched chunk #15 = documents up to #64000/126225, outstanding queue size 16
INFO : PROGRESS: pass 18, dispatched chunk #16 = documents up to #68000/126225, outstanding queue size 17
INFO : PROGRESS: pass 18, dispatched chunk #17 = documents up to #72000/126225, outstanding queue size 18
INFO : PROGRESS: pass 18, dispatched chunk #18 = documents up to #76000/126225, outstanding queue size 19
INFO : PROGRESS: pass 18, dispatched chunk #19 = documents up to #80000/126225, outstanding queue size 20
INFO : PROGRESS: pass 18, dispatched chunk #20 = documents up to #84000/126225, outstanding queue size 21
INFO : PROGRESS: pass 18, dispatched chunk #21 =

DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : processing chunk #31 of 2225 documents
DEBUG : performing inference on a chunk of 2225 documents
DEBUG : processed chunk, queuing the result
DEBUG : 3570/4000 documents converged within 50 iterations
DEBUG : result put
DEBUG : getting a new job
DEBUG : 3540/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : 3561/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : result put
DEBUG : getting a new job
DEBUG : getting a new job
DEBUG : 3530/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : 3541/4000 documents converged within 50 iterations
DEBUG : result put
DEBUG : processed chunk, queuing the result
DEBUG : getting a new job
DEBUG : 3591/4000 documents converged within 50 iterations
DEBUG : result put
DEBUG : proces

DEBUG : performing inference on a chunk of 4000 documents
DEBUG : 3557/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : 3552/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : processing chunk #13 of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : processing chunk #14 of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : 3572/4000 documents converged within 50 iterations
DEBUG : processed chunk, queuing the result
DEBUG : result put
DEBUG : getting a new job
DEBUG : processing chunk #15 of 4000 documents
DEBUG : processing chunk #16 of 4000 documents
DEBUG : processed chunk, queuing the result
DEBUG : performing inference on a chunk of 4000 documents
DEBUG : 3563/4000 documents converged within 50 iterations
D

In [57]:
lda.save('lda_final2')
dictionary.save('dictionary2')
corpora.MmCorpus.serialize('doc_term_matrix.mm2', doc_term_matrix)

INFO : saving LdaState object under lda_final2.state, separately None
DEBUG : {'kw': {}, 'mode': 'wb', 'uri': 'lda_final2.state'}
DEBUG : encoding_wrapper: {'errors': 'strict', 'encoding': None, 'mode': 'wb', 'fileobj': <_io.BufferedWriter name='lda_final2.state'>}
INFO : saved lda_final2.state
DEBUG : {'kw': {}, 'mode': 'wb', 'uri': 'lda_final2.id2word'}
DEBUG : encoding_wrapper: {'errors': 'strict', 'encoding': None, 'mode': 'wb', 'fileobj': <_io.BufferedWriter name='lda_final2.id2word'>}
INFO : saving LdaMulticore object under lda_final2, separately ['expElogbeta', 'sstats']
INFO : storing np array 'expElogbeta' to lda_final2.expElogbeta.npy
INFO : not storing attribute dispatcher
INFO : not storing attribute id2word
INFO : not storing attribute state
DEBUG : {'kw': {}, 'mode': 'wb', 'uri': 'lda_final2'}
DEBUG : encoding_wrapper: {'errors': 'strict', 'encoding': None, 'mode': 'wb', 'fileobj': <_io.BufferedWriter name='lda_final2'>}
INFO : saved lda_final2
INFO : saving Dictionary ob